In [8]:
from elasticsearch import Elasticsearch, helpers
import json

In [9]:
from indexing.enums.index_names import IndexName
from indexing.mappings.mappings import ESMappings

In [13]:
# Password for the 'elastic' user generated by Elasticsearch
from shared.configs.elastic_search_config import ELASTIC_PASSWORD

# Create the client instance
es = Elasticsearch(
    "https://localhost:9200",
    ca_certs="config/certs/http_ca.crt",
    basic_auth=("elastic", ELASTIC_PASSWORD),
    verify_certs=False
)

C:\Users\nicki\AppData\Roaming\Python\Python38\site-packages\elasticsearch\_sync\client\__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [14]:
es.info()

C:\Users\nicki\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'name': 'DESKTOP-5628FPD', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'kfVRx5jQRzeDrEWE0UZmog', 'version': {'number': '8.6.1', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '180c9830da956993e59e2cd70eb32b5e383ea42c', 'build_date': '2023-01-24T21:35:11.506992272Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Create BM25 index

In [ ]:
# create index
index_name = IndexName.UWATERLOO_COURSES_INDEX.value
mapping = ESMappings.get_mapping_from_index_name(index_name)
es.indices.create(index=index_name, body=mapping)

In [ ]:
# load data
with open("../scraping/contents/waterloo/output.json", "r") as f:
    data = json.load(f)

In [ ]:
def data_gen(data: list):
    for item in data:
        yield {
            "_index": IndexName.UWATERLOO_COURSES_INDEX,
            "_source": item
        }

In [ ]:
helpers.bulk(es, data_gen(data))

In [ ]:
es.search(index=IndexName.UWATERLOO_COURSES_INDEX.value, body={"from": 0, "size": 1, "query": {"match": {"courseDescription": "Machine Intelligence"}}})

In [16]:
es.indices.create(index=IndexName.UWATERLOO_COURSES_INDEX_DPR.value)

C:\Users\nicki\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'uwaterloo-courses-dpr'})

In [15]:
es.indices.delete(index=IndexName.UWATERLOO_COURSES_INDEX.value)

C:\Users\nicki\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [uwaterloo-courses-dpr]', uwaterloo-courses-dpr, index_or_alias)

In [20]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
query = "Machine Intelligence"
tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
model = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
input_ids = tokenizer(query, return_tensors="pt")["input_ids"]
embeddings = model(input_ids).pooler_output.tolist()


In [25]:
es.search(index=IndexName.UWATERLOO_COURSES_INDEX_DPR.value, body={"from": 0, "size": 1, "query": {"script": {"source": "cosineSimilarity(params.queryVector, 'courseDescEncoding') + 1.0", 
                                                                                                    "params": {"queryVector": embeddings}
                                                                                                    }}})

C:\Users\nicki\AppData\Local\Temp\ipykernel_3296\3047621199.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.search(index=IndexName.UWATERLOO_COURSES_INDEX_DPR.value, body={"from": 0, "size": 1, "query": {"script": {"source": "cosineSimilarity(params.queryVector, 'courseDescEncoding') + 1.0",
C:\Users\nicki\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


BadRequestError: BadRequestError(400, 'parsing_exception', '[script] query does not support [source]')

In [ ]:
es.indices.delete(index=IndexName.UWATERLOO_COURSES_INDEX_DPR.value)